# Run all default adverSCarial attacks on chosen clusters, on all datasets, on all classifiers, and store compute time.

In [1]:
lapply(c("dplyr","Seurat","HGNChelper", "adverSCarial", "IRdisplay"), library, character.only = T)
options(warn=-1)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




[[1]]
[1] "dplyr"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

[[2]]
 [1] "Seurat"       "SeuratObject" "sp"           "dplyr"        "stats"       
 [6] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[11] "base"        

[[3]]
 [1] "HGNChelper"   "Seurat"       "SeuratObject" "sp"           "dplyr"       
 [6] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[11] "methods"      "base"        

[[4]]
 [1] "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject" "sp"          
 [6] "dplyr"        "stats"        "graphics"     "grDevices"    "utils"       
[11] "datasets"     "methods"      "base"        

[[5]]
 [1] "IRdisplay"    "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject"
 [6] "sp"           "dplyr"        "stats"        "graphics"     "grDevices"   
[11] "utils"        "datasets"     "methods"      "base"

In [2]:
bd <- "data/"
my_computer= "c2"

source("load_classifiers_v5.r")

In [3]:
c_paths <- list(pbmc_3k=list(so=paste0(bd, "v2//pbmc_test.rds"),
                             sce=paste0(bd, "v2//sce_pbmc_test.rds"),
                             df=paste0(bd, "v2//seurat_scaled_pbmc_test.txt")),
               kidney_10k=list(so=paste0(bd, "v5/data/sc/hgnc_kidney_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_kidney_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_kidney_10k_test.txt")),
                liver_10k=list(so=paste0(bd, "v5/data/sc/hgnc_liver_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_liver_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_liver_10k_test.txt")),
                axilla_10k=list(so=paste0(bd, "v5/data/sc/hgnc_axilla_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_axilla_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_axilla_10k_test.txt"))
               )

In [4]:
lsg <- list(pbmc_3k= read.table("data//v2/sign_genes.txt")$gene,
           liver_10k = read.table("data//v5/data/sign/sign_liver_10k.txt")$gene,
           kidney_10k = read.table("data//v5/data/sign/sign_kidney_10k.txt")$gene,
           axilla_10k = read.table("data//v5/data/sign/sign_axilla_10k.txt")$gene)

In [5]:
c_corres <- readRDS("data//v5/type_corres.rds")

In [6]:
unique(unlist(lapply(grep("pbmc",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))

[1] "Memory CD4 T" "B"            "CD14+ Mono"   "NK"           "CD8 T"       
[6] "Naive CD4 T"  "FCGR3A+ Mono" "DC"           "Platelet"

In [7]:
unique(unlist(lapply(grep("axilla",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))

[1] "malignant cell"                  "blood vessel endothelial cell"  
[3] "fibroblast"                      "macrophage"                     
[5] "blood vessel smooth muscle cell" "T cell"

In [8]:
unique(unlist(lapply(grep("liver",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))

[1] "malignant cell"                  "endothelial cell"               
[3] "blood vessel smooth muscle cell" "macrophage"                     
[5] "monocyte"                        "mature NK T cell"

In [9]:
unique(unlist(lapply(grep("kidney",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))

[1] "kidney epithelial cell"              
[2] "kidney cell"                         
[3] "podocyte"                            
[4] "epithelial cell of proximal tubule"  
[5] "mesenchymal cell"                    
[6] "kidney loop of Henle epithelial cell"

In [10]:
# list of attacks to run
c_ids2run = c(
 "CGD;MLP;liver_10k;macrophage;0.5;0.5;5000",
 "CGD;MLP;pbmc_3k;CD8 T;10;10;5000",
 "CGD;MLP;axilla_10k;blood vessel smooth muscle cell;10;10;5000",
 "CGD;MLP;kidney_10k;mesenchymal cell;10;10;5000",
 "CGD;MLP;liver_10k;macrophage;10;10;5000",
"singleGene;scAnnotatR;axilla_10k;fibroblast;perc99;T",
    "singleGene;scAnnotatR;liver_10k;endothelial cell;perc99;T",
    
    "singleGene;scAnnotatR;axilla_10k;macrophage;perc1;T",
    "singleGene;scAnnotatR;liver_10k;monocyte;perc1;T",
    
    "singleGene;scAnnotatR;axilla_10k;blood vessel smooth muscle cell;positive_aberrant;T",
    "singleGene;scAnnotatR;liver_10k;macrophage;positive_aberrant;T",
    
    "singleGene;scAnnotatR;axilla_10k;malignant cell;negative_aberrant;T",
    "singleGene;scAnnotatR;liver_10k;malignant cell;negative_aberrant;T",
    
    "singleGene;scAnnotatR;axilla_10k;fibroblast;positive_aberrant;T",
    "singleGene;scAnnotatR;liver_10k;endothelial cell;positive_aberrant;T",
        
    "singleGene;CHETAH;axilla_10k;fibroblast;positive_aberrant;T",
    "singleGene;CHETAH;liver_10k;endothelial cell;positive_aberrant;T",
    
    "singleGene;CHETAH;axilla_10k;macrophage;positive_aberrant;T",
    "singleGene;CHETAH;liver_10k;monocyte;positive_aberrant;T",
    
    "singleGene;CHETAH;axilla_10k;blood vessel smooth muscle cell;positive_aberrant;T",
    "singleGene;CHETAH;liver_10k;macrophage;positive_aberrant;T",
    
    "singleGene;CHETAH;axilla_10k;malignant cell;positive_aberrant;T",
    "singleGene;CHETAH;liver_10k;malignant cell;positive_aberrant;T",
    
    
    "singleGene;CHETAH;axilla_10k;fibroblast;negative_aberrant;T",
    "singleGene;CHETAH;liver_10k;endothelial cell;negative_aberrant;T",
    
    "singleGene;CHETAH;axilla_10k;macrophage;negative_aberrant;T",
    "singleGene;CHETAH;liver_10k;monocyte;negative_aberrant;T",
    
    "singleGene;CHETAH;axilla_10k;blood vessel smooth muscle cell;negative_aberrant;T",
    "singleGene;CHETAH;liver_10k;macrophage;negative_aberrant;T",
    
    "singleGene;CHETAH;axilla_10k;malignant cell;negative_aberrant;T",
    "singleGene;CHETAH;liver_10k;malignant cell;negative_aberrant;T",
        
    "singleGene;CHETAH;axilla_10k;fibroblast;perc99;T",
    "singleGene;CHETAH;liver_10k;endothelial cell;perc99;T",
    
    "singleGene;CHETAH;axilla_10k;macrophage;perc1;T",
    "singleGene;CHETAH;liver_10k;monocyte;perc1;T",
    
    "singleGene;CHETAH;axilla_10k;blood vessel smooth muscle cell;positive_aberrant;T",
    "singleGene;CHETAH;liver_10k;macrophage;positive_aberrant;T",
    
    "singleGene;CHETAH;axilla_10k;malignant cell;negative_aberrant;T",
    "singleGene;CHETAH;liver_10k;malignant cell;negative_aberrant;T",
    "singleGene;CHETAH;kidney_10k;kidney cell;perc99;T",
    
    "singleGene;scType;pbmc_3k;NK;perc1;F",
    "singleGene;scType;axilla_10k;malignant cell;perc1;F",
    "singleGene;scType;liver_10k;blood vessel smooth muscle cell;perc1;F",
    "singleGene;scType;kidney_10k;kidney epithelial cell;perc1;F",
    
    "maxChange;RF;pbmc_3k;Memory CD4 T;decile+5;1000",
    "maxChange;RF;axilla_10k;blood vessel endothelial cell;decile+5;1000",
    "maxChange;RF;liver_10k;malignant cell;decile+5;1000",
    "maxChange;RF;kidney_10k;podocyte;decile+5;1000",
    
    "maxChange;RF;pbmc_3k;Memory CD4 T;perc99;1000",
    "maxChange;RF;axilla_10k;blood vessel endothelial cell;perc99;1000",
    "maxChange;RF;liver_10k;malignant cell;perc99;1000",
    "maxChange;RF;kidney_10k;podocyte;perc99;1000",
    
    "maxChange;MLP;pbmc_3k;CD14+ Mono;negative_aberrant;1",
    "maxChange;MLP;axilla_10k;macrophage;negative_aberrant;1",
    "maxChange;MLP;liver_10k;monocyte;negative_aberrant;1",
    "maxChange;MLP;kidney_10k;epithelial cell of proximal tubule;negative_aberrant;1",
    
    "CGD;scAnnotatR;pbmc_3k;CD8 T;0.5;0.5;500",
    "CGD;scAnnotatR;axilla_10k;blood vessel smooth muscle cell;0.5;0.5;500",
    "CGD;scAnnotatR;liver_10k;macrophage;0.5;0.5;500",
    "CGD;scAnnotatR;kidney_10k;mesenchymal cell;0.5;0.5;500",
    
    "CGD;MLP;pbmc_3k;Naive CD4 T;1;1;500",
    "CGD;MLP;axilla_10k;T cell;1;1;500",
    "CGD;MLP;liver_10k;mature NK T cell;1;1;500",
    "CGD;MLP;kidney_10k;kidney loop of Henle epithelial cell;1;1;500",
    
    "CGD;RF;pbmc_3k;FCGR3A+ Mono;10;10;500",
    "CGD;RF;axilla_10k;malignant cell;10;10;500",
    "CGD;RF;liver_10k;endothelial cell;10;10;500",
    "CGD;RF;kidney_10k;kidney epithelial cell;10;10;500"
)
    
c_ids2run <- c(paste0("c1;",c_ids2run), paste0("c2;", c_ids2run[length(c_ids2run):1]))
c_ids2run

[1] "c1;CGD;MLP;liver_10k;macrophage;0.5;0.5;5000"                    
 [2] "c1;CGD;MLP;pbmc_3k;CD8 T;10;10;5000"                             
 [3] "c1;CGD;MLP;axilla_10k;blood vessel smooth muscle cell;10;10;5000"
 [4] "c1;CGD;MLP;kidney_10k;mesenchymal cell;10;10;5000"               
 [5] "c1;CGD;MLP;liver_10k;macrophage;10;10;5000"                      
 [6] "c2;CGD;MLP;liver_10k;macrophage;0.5;0.5;5000"                    
 [7] "c2;CGD;MLP;pbmc_3k;CD8 T;10;10;5000"                             
 [8] "c2;CGD;MLP;axilla_10k;blood vessel smooth muscle cell;10;10;5000"
 [9] "c2;CGD;MLP;kidney_10k;mesenchymal cell;10;10;5000"               
[10] "c2;CGD;MLP;liver_10k;macrophage;10;10;5000"

In [11]:
df_times <- readRDS("data//v5/df_times.rds")
df_times

,modifs,classifier,ct,ds,run_time,misc
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
c1;singleGene;CHETAH;pbmc_3k;B;perc99;T,-1,-1,-1,-1,00:47:14,0;
c1;singleGene;CHETAH;axilla_10k;fibroblast;perc99;T,-1,-1,-1,-1,-1,-1
c1;singleGene;CHETAH;liver_10k;endothelial cell;perc99;T,-1,-1,-1,-1,-1,-1
c1;singleGene;CHETAH;kidney_10k;kidney cell;perc99;T,-1,-1,-1,-1,-1,-1
c1;singleGene;scType;pbmc_3k;NK;perc1;F,-1,-1,-1,-1,00:03:39,0;0
c1;singleGene;scType;axilla_10k;malignant cell;perc1;F,-1,-1,-1,-1,02:23:16,0;0
c1;singleGene;scType;liver_10k;blood vessel smooth muscle cell;perc1;F,-1,-1,-1,-1,02:59:40,1;1
c1;singleGene;scType;kidney_10k;kidney epithelial cell;perc1;F,-1,-1,-1,-1,04:24:06,2;1
c1;maxChange;RF;pbmc_3k;Memory CD4 T;decile+5;1000,-1,-1,-1,-1,00:02:53,13714;0;1318;13714


In [12]:
nchar(as.character(df_times["c2;singleGene;scType;kidney_10k;kidney epithelial cell;perc1;F", "run_time"]))

[1] 8

In [13]:
nchar(as.character(df_times["c2;singleGene;scType;liver_10k;blood vessel smooth muscle cell;perc1;F", "run_time"]))

[1] 8

In [14]:
myHourFormat <- function(start_time){
    diff_secs <- as.numeric(difftime(Sys.time(),start_time, units = "secs"))

    # Extraction des heures, minutes et secondes
    heures <- floor(diff_secs / 3600)
    minutes <- floor((diff_secs %% 3600) / 60)
    secondes <- round(diff_secs %% 60)

    # Formatage du temps écoulé
    temps_formate <- sprintf("%02d:%02d:%02d", heures, minutes, secondes)
    return(temps_formate)    
}


In [15]:
exportResults <- function(acbc, baseDir, expID, strClassifier){
    write.table(acbc$byGeneSummary,
        paste0(baseDir,"/byGeneSummary_",expID,"stopAtSwitch=F.txt"), row.names=F)
    display(acbc$oneRowSummary)
    dfOneRowSummary <- as.data.frame(matrix(nrow=0, ncol=17))
    dfOneRowSummary <- rbind(dfOneRowSummary, c(strClassifier, acbc$oneRowSummary))
    colnames(dfOneRowSummary) <- c("classifier", "cluster","cluster_size", "targetCluster",
                                  "alpha","epsilon", "changeClust",
                                  "nbModifiedGenes", "nbTestedGenes",
                                  "oriPourcStart", "targPourcStart",
                                  "oriPourcEnd", "targPourcEnd",
                                   "wholeClust_oriOdd_beg", "wholeClust_targOdd_beg",
                                  "wholeClust_oriOdd_end", "wholeClust_targOdd_end")
    write.table(dfOneRowSummary,
                paste0(baseDir,"oneRowSummary_",expID,"stopAtSwitch=F.txt"),
                row.names=F)
}

In [16]:
only_myComp_ids <- grep(paste0("^",my_computer), c_ids2run, value=T)
only_myComp_ids

[1] "c2;CGD;MLP;liver_10k;macrophage;0.5;0.5;5000"                    
[2] "c2;CGD;MLP;pbmc_3k;CD8 T;10;10;5000"                             
[3] "c2;CGD;MLP;axilla_10k;blood vessel smooth muscle cell;10;10;5000"
[4] "c2;CGD;MLP;kidney_10k;mesenchymal cell;10;10;5000"               
[5] "c2;CGD;MLP;liver_10k;macrophage;10;10;5000"

In [17]:
only_myComp_ids <- grep(paste0("^",my_computer), c_ids2run, value=T)
for (str_id in only_myComp_ids){
    a_a = unlist(strsplit(str_id, ";"))
#     my_comp <- a_a[1]
    myAttack <- a_a[2]

    baseClass <- a_a[3]
    basen <- a_a[4]
    myCluster <- a_a[5]
    param1 <- a_a[6]
    param2 <- a_a[7]
    param3 <- a_a[8]
    
    if (!str_id %in% names(df_times)){
        df_times[str_id,] = -1
    }
    if (nchar(as.character(df_times[str_id,"run_time"])) == 2){
        display(str_id)
# for (a_a in c_atts){
#     basen = a_a[1]
#     baseClass = a_a[2]
#     modif = a_a[3]
#     myCluster = a_a[4]
# for (basen in c_b2){
        message(basen)
        my_path <- paste0(bd, "v5/data/sc/hgnc_", basen, "_test.txt")
        my_path <- c_paths[[basen]]$df
        if (!file.exists(my_path)){
            message(my_path, " not available")
            next
        }
        df_pbmc_test <- read.table(my_path)
        expr_df <- df_pbmc_test[, -which(names(df_pbmc_test) == "y")]
        clusters_df <- df_pbmc_test$y
        names(clusters_df) <- rownames(df_pbmc_test)

#         sign_path <- paste0(paste0(bd, "v5/data/sign/sign_", basen, ".txt"))
#         if (!file.exists(sign_path)){
#             message(sign_path, " not available, computing getSignGenes")
#             dfSG <- getSignGenes(expr_df, clusters_df, verbose=T)
#             write.table(dfSG$results, sign_path)
#         }
#         mySignGenes <- read.table(sign_path)

        message(baseClass)
        strClassifier <- paste0(baseClass, "_", basen)
        if ( baseClass == "scType" || basen=="pbmc_3k"){
            strClassifier <- baseClass
        }
        display(strClassifier)
        myClassifier <- classifList[[strClassifier]]$fct
        inputType <- classifList[[strClassifier]]$inputType

        display(inputType)
        mySlot <- classifList[[strClassifier]]$slot
        expr <- ""

        if(inputType == "data.frame"){
            expr <- expr_df
            clusters <- clusters_df
        }
        if(tolower(inputType) == "seurat"){
            my_path <- paste0(bd, "v5/data/sc/hgnc_", basen, "_test.rds")
            my_path <- c_paths[[basen]]$so
            if (!file.exists(my_path)){
                message(my_path, " not available")
                break
            }
            so_pbmc_test <- readRDS(my_path)

            clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
            names(clusters_so) <- rownames(so_pbmc_test@meta.data)

            expr <- so_pbmc_test
            clusters <- clusters_so
        }
        if (tolower(inputType) == "singlecellexperiment"){
            my_path1 <- paste0(bd, "v5/data/sc/hgnc_",basen,"_test.rds")
            my_path1 <- c_paths[[basen]]$so
            my_path2 <- paste0(bd, "v5/data/sc/sce_hgnc_",basen,"_test.rds")
            my_path2 <- c_paths[[basen]]$sce
            if (!file.exists(my_path1) || !file.exists(my_path2)){
                message(my_path1, " or ", my_path2," not available")
                break
            }
            so_pbmc_test <- readRDS(my_path1)
            clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
            names(clusters_so) <- rownames(so_pbmc_test@meta.data)

            sce_pbmc_test <- readRDS(my_path2)
            clusters_sce = so_pbmc_test@meta.data$chr_seurat_cluster
            names(clusters_sce) <- rownames(so_pbmc_test@meta.data)

            expr <- sce_pbmc_test
            clusters <- clusters_sce
        }
        message(myCluster)
        
        if (myAttack == "CGD"){
            alpha <- as.numeric(param1)
            epsilon <- as.numeric(param2)
            nGenes <- as.numeric(param3)
            
            my_clusters = as.character(unname(clusters))
            if ( inputType == "SingleCellExperiment"){
                my_clusters = as.character(expr@colData$chr_seurat_cluster)
            }
            
            expID <- paste0(strClassifier,"_cellByCell_alpha=",alpha,"_epsilon=",
                            epsilon, "__",stringr::str_replace_all(myCluster," ","_"),"__", basen, "__nGenes=",
                           nGenes)
            filePath <- paste0(bd,"v2/results/","oneRowSummary_",expID,".txt")
            message(filePath)
            
            t1 <- Sys.time()
            acbc <- advCGD(expr, my_clusters, myCluster, myClassifier,
                              genes=lsg[[basen]][1:nGenes],
                                 exclNewTargets=NULL, newTarget=NULL,
                                 alpha=alpha, epsilon=epsilon,
                                 slot=mySlot, stopAtSwitch=T, verbose=T)
            
            df_times <- readRDS("data//v5/df_times.rds")
            df_times[str_id, "run_time"] <- myHourFormat(t1)
            
            dfOneRowSummary <- as.data.frame(matrix(nrow=0, ncol=17))
            dfOneRowSummary <- rbind(dfOneRowSummary, c(strClassifier, acbc$oneRowSummary))
            colnames(dfOneRowSummary) <- c("classifier", "cluster","cluster_size", "targetCluster",
                                          "alpha","epsilon", "changeClust",
                                          "nbModifiedGenes", "nbTestedGenes",
                                          "oriPourcStart", "targPourcStart",
                                          "oriPourcEnd", "targPourcEnd",
                                           "wholeClust_oriOdd_beg", "wholeClust_targOdd_beg",
                                          "wholeClust_oriOdd_end", "wholeClust_targOdd_end")
            if (dfOneRowSummary[1,"oriPourcEnd"] < dfOneRowSummary[1,"targPourcEnd"]){
                str_misc <- "T;"
                
            } else {
                str_misc <- "F;"
            }
            str_misc <- paste0(str_misc, dfOneRowSummary[1,"targetCluster"], ";")
            
            mod_genes <- sum(acbc$byGeneSummary[,"nbModifiedCells"] != 0) - 1
            str_misc <- paste0(str_misc, mod_genes, ";")
            str_misc <- paste0(str_misc, nrow(acbc$byGeneSummary))
            df_times[str_id, "misc"] <- str_misc
                                                  
            saveRDS(df_times, "data//v5/df_times.rds")

             if (!file.exists(filePath)){
                exportResults(acbc, paste0(bd,"v2/results/"), expID, strClassifier)
            } else {
                display(paste(expID, "already exported"))
            }
        }
        
        if (myAttack == "maxChange"){
            modif <- param1
            my_clusters = as.character(unname(clusters))
            if ( inputType == "SingleCellExperiment"){
                my_clusters = as.character(expr@colData$chr_seurat_cluster)
            }
            path_genes <- paste0(bd,"v2/results/",strClassifier,"_maxChange_",modif,"__",myCluster,"__",basen,"__maxSplitSize=",param2,".txt")
            path_expr <- paste0(bd,"v2/im/temp_data/df_",strClassifier,"_maxChange_",modif,"__",myCluster,"__",basen,"__maxSplitSize=",param2,".txt")
            
            t1 <- Sys.time()
            max_temp = advMaxChange(expr,
                                    my_clusters,
                                    myCluster,
                                    myClassifier, verbose=T,
                                    argForClassif=inputType,
                                    argForModif=inputType,
                                    advMethod=modif,
                                    slot=mySlot,
                                   maxSplitSize=as.numeric(param2))
            
            df_times <- readRDS("data//v5/df_times.rds")
            df_times[str_id, "run_time"] <- myHourFormat(t1)

            signa <- max(nrow(expr), ncol(expr)) - length(max_temp@values)
            
            str_misc <- paste0(length(max_temp@values), ";",
                               signa, ";",
                               nrow(expr),";",ncol(expr)
                            )

            df_times[str_id, "misc"] <- str_misc
                                                  
            saveRDS(df_times, "data//v5/df_times.rds")
            if (!file.exists(path_genes)){
                write.table(max_temp@values, path_genes)
            } else {
                display(paste(path_genes, " already exists"))
            }
        }

        if ( myAttack == "singleGene"){
            modif <- param1
            filePath <- paste0(bd,"v2/results/",strClassifier,"_singleGene_",modif,"__",
                               stringr::str_replace_all(myCluster," ","_"),"__", basen, "__",param2,".txt")
            message(filePath)
            myFirstDichot = 100
            my_clusters = as.character(unname(clusters))
            if ( inputType == "SingleCellExperiment"){
                my_clusters = as.character(expr@colData$chr_seurat_cluster)
            }

            t1 <- Sys.time()

            min_temp = advSingleGene(expr,
                                     my_clusters,
                                      myCluster,
                                     myClassifier, verbose=T,
                                     argForClassif=inputType,
                                     argForModif=inputType,
                                     advMethod=modif,slot=mySlot,
                                     firstDichot=myFirstDichot,
                                     returnFirstFound= param2=="T")

            df_times <- readRDS("data//v5/df_times.rds")
            if (!str_id %in% names(df_times)){
                df_times[str_id,] = -1
            }
            df_times[str_id,"run_time"] <- myHourFormat(t1)
            if (param2 == "T"){
                str_misc <- paste0(length(names(min_temp@values)),";",
                      unname(unlist(lapply(min_temp@values, function(x) x[1]))))

            } else {
                str_misc <- paste0(length(names(min_temp@values)),";",
                      length(unique(unname(unlist(lapply(min_temp@values, function(x) x[1])))))
                )
            }
            display(min_temp)
            display(min_temp@values)
            df_times[str_id,"misc"] <- str_misc
            saveRDS(df_times, "data//v5/df_times.rds")
            
            if (!file.exists(filePath)){
                df_4export = data.frame(genes = names(min_temp@values),
                       cell_type=unname(unlist(lapply(min_temp@values, function(x) x[1]))))
                write.table(df_4export, filePath)
            } else {
                display(paste(filePath, "already exported"))
            }
            
        }
    } else {
        display(paste(str_id, " already done"))
    }
}


[1] "c2;CGD;MLP;liver_10k;macrophage;0.5;0.5;5000"

liver_10k

MLP



[1] "MLP_liver_10k"

[1] "data.frame"

macrophage

data/v2/results/oneRowSummary_MLP_liver_10k_cellByCell_alpha=0.5_epsilon=0.5__macrophage__liver_10k__nGenes=5000.txt

macrophage

50006

5000

205

New cluster target: monocyte

VWA1 1

Number of original annot macrophage : 203

 mean 0.993501969452562 delt 0

Number of monocyte : 2

 mean 0.711219877004623 delt 0

Number of modified cells 203

AOAH 2

Number of original annot macrophage : 203

 mean 0.993486811374796 delt -1.51580777660865e-05

Number of monocyte : 2

 mean 0.711219877004623 delt 0

Number of modified cells 203

CD247 3

Number of original annot macrophage : 203

 mean 0.993348971669897 delt -0.000137839704898779

Number of monocyte : 2

 mean 0.711219877004623 delt 0

Number of modified cells 203

AKT3 4

Number of original annot macrophage : 203

 mean 0.993239984430116 delt -0.000108987239781255

Number of monocyte : 2

 mean 0.711219877004623 delt 0

Number of modified cells 203

PTPRC 5

Number of original annot macrophage : 203

 mean 0.9932111281479

...

[1] "MLP_liver_10k"

[1] "data.frame"

macrophage

data/v2/results/oneRowSummary_MLP_liver_10k_cellByCell_alpha=10_epsilon=10__macrophage__liver_10k__nGenes=5000.txt

macrophage

50006

5000

205

New cluster target: monocyte

VWA1 1

Number of original annot macrophage : 203

 mean 0.993501969452562 delt 0

Number of monocyte : 2

 mean 0.711219877004623 delt 0

Number of modified cells 203

AOAH 2

Number of original annot macrophage : 203

 mean 0.993176273230849 delt -0.000325696221713301

Number of monocyte : 2

 mean 0.711219877004623 delt 0

Number of modified cells 203

CD247 3

Number of original annot macrophage : 203

 mean 0.989080484864747 delt -0.00409578836610169

Number of monocyte : 2

 mean 0.711219877004623 delt 0

Number of modified cells 203

AKT3 4

Number of original annot macrophage : 202

 mean 0.990084866485973 delt 0.0010043816212264

Number of monocyte : 3

 mean 0.725147982438405 delt 0.013928105433782

Number of modified cells 202

PTPRC 5

Number of original annot macrophage : 202

 mean 0.989

Number of monocyte : 56

 mean 0.594850465123143 delt -0.00882353245907896

Number of modified cells 149

CPVL 42

Number of original annot macrophage : 116

 mean 0.969813962948733 delt 0.00447875354850746

Number of monocyte : 56

 mean 0.594850465123143 delt 0

Number of modified cells 149

PLCL1 43

Number of original annot macrophage : 112

 mean 0.965183942711779 delt -0.00463002023695425

Number of monocyte : 57

 mean 0.591106609294289 delt -0.0037438558288535

Number of modified cells 148

TGFBR3 44

Number of original annot macrophage : 111

 mean 0.965687197070938 delt 0.000503254359158878

Number of monocyte : 57

 mean 0.591106609294289 delt 0

Number of modified cells 148

RCSD1 45

Number of original annot macrophage : 110

 mean 0.969146558913318 delt 0.0034593618423796

Number of monocyte : 58

 mean 0.58959974502695 delt -0.00150686426733881

Number of modified cells 147

COBLL1 46

Number of original annot macrophage : 110

 mean 0.967835712703792 delt -0.00131084620

[1] "macrophage"          "205"                 "monocyte"           
 [4] "10"                  "10"                  "0"                  
 [7] "65"                  "65"                  "0.990243902439024"  
[10] "0.00975609756097561" "0.351219512195122"   "0.370731707317073"  
[13] NA                    NA                    NA                   
[16] NA